In [14]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [15]:
perfil = pd.read_csv('../data/clear_perfil_visitor_and_students.csv')

In [16]:
perfil.columns

Index(['Unnamed: 0', 'email', 'lead_scoring_-_perfil', 'url_pública',
       'estágio_no_funil', 'total_de_conversões', 'lead_scoring_-_interesse',
       'status_para_comunicação_por_email', 'data_da_primeira_conversão',
       'eventos_(últimos_100)', 'origem_da_última_conversão',
       'data_da_última_conversão', 'origem_da_primeira_conversão', 'empresa',
       'tags', 'data_da_última_oportunidade', 'estado',
       'valor_total_da_oportunidade_no_crm_(última_atualização)',
       'qualificação_da_oportunidade_no_crm_(última_atualização)',
       'etapa_do_funil_de_vendas_no_crm_(última_atualização)',
       'nome_do_responsável_pela_oportunidade_no_crm_(última_atualização)',
       'origem_da_oportunidade_no_crm_(última_atualização)',
       'possui_interesse_em_realizar_uma_pós-graduação_ou_mba?',
       'desejo_receber_o_sustentare_news', 'como_você_conheceu_a_sustentare:',
       'universidade', 'qual_o_curso_de_interesse?', 'cargo_final',
       'area_atuacao', 'interesse_fin

In [17]:
df = perfil[['lead_scoring_-_perfil', 'estágio_no_funil', 'total_de_conversões', 'lead_scoring_-_interesse', 
                 'status_para_comunicação_por_email','origem_da_última_conversão', 'origem_da_primeira_conversão', 
                 'empresa','estado', 'valor_total_da_oportunidade_no_crm_(última_atualização)', 
                 'qualificação_da_oportunidade_no_crm_(última_atualização)', 'nome_do_responsável_pela_oportunidade_no_crm_(última_atualização)',
                 'origem_da_oportunidade_no_crm_(última_atualização)', 'possui_interesse_em_realizar_uma_pós-graduação_ou_mba?', 
                 'desejo_receber_o_sustentare_news', 'como_você_conheceu_a_sustentare:', 'universidade','qual_o_curso_de_interesse?',
                 'cargo_final', 'area_atuacao', 'interesse_final', 'aluno', 'cidade', 'months_since_first_conversion',
                 'months_since_last_conversion', 'months_since_last_opportunity', 'age']]

In [18]:
df.dtypes

lead_scoring_-_perfil                                                 object
estágio_no_funil                                                      object
total_de_conversões                                                    int64
lead_scoring_-_interesse                                               int64
status_para_comunicação_por_email                                       bool
origem_da_última_conversão                                            object
origem_da_primeira_conversão                                          object
empresa                                                               object
estado                                                                object
valor_total_da_oportunidade_no_crm_(última_atualização)              float64
qualificação_da_oportunidade_no_crm_(última_atualização)             float64
nome_do_responsável_pela_oportunidade_no_crm_(última_atualização)     object
origem_da_oportunidade_no_crm_(última_atualização)                    object

In [19]:
perfil['cidade'].value_counts()

cidade
Joinville             4288
Jaraguá do Sul         155
São Paulo              128
Jaragua do sul         126
Sao paulo              117
                      ... 
Primavera do Leste       1
Aperibé                  1
Dois Irmãos              1
Nova Friburgo            1
Boa vista                1
Name: count, Length: 426, dtype: int64

In [10]:
df[df.select_dtypes(['object']).columns] = df.select_dtypes(['object']).apply(lambda x:x.astype(int))

ValueError: invalid literal for int() with base 10: 'd'

In [30]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = df.drop('aluno', axis=1)
y = df['aluno']

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the XGBoost classifier
xgb_clf = xgb.XGBClassifier(objective='binary:logistic', missing=None, seed=42, enable_categorical =True)

# Train the classifier
xgb_clf.fit(X_train, y_train)

# Predictions
y_pred = xgb_clf.predict(X_test)

accuracy = accuracy_score(y_test,y_pred)

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:lead_scoring_-_perfil: object, estágio_no_funil: object, origem_da_última_conversão: object, origem_da_primeira_conversão: object, empresa: object, estado: object, nome_do_responsável_pela_oportunidade_no_crm_(última_atualização): object, origem_da_oportunidade_no_crm_(última_atualização): object, possui_interesse_em_realizar_uma_pós-graduação_ou_mba?: object, como_você_conheceu_a_sustentare:: object, universidade: object, qual_o_curso_de_interesse?: object, cargo_final: object, area_atuacao: object, interesse_final: object, cidade: object

In [12]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming df is your DataFrame

# Convert object columns to categorical
categorical_columns = df.select_dtypes(include=['object']).columns
df[categorical_columns] = df[categorical_columns].astype('category')

# Extract features and target variable
X = df.drop('aluno', axis=1)
y = df['aluno']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the XGBoost classifier with enable_categorical=True
xgb_clf = xgb.XGBClassifier(objective='binary:logistic', missing=None, seed=42, enable_categorical=True)

# Train the classifier
xgb_clf.fit(X_train, y_train)

# Make predictions
y_pred = xgb_clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")


XGBoostError: [17:13:58] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0750514818a16474a-1\xgboost\xgboost-ci-windows\include\xgboost/json.h:630: Invalid type for: `missing`, expecting one of the: {``Number`, `Integer`}, got: `Null`